# EDA 5번) 대구의 지형 특성 (분지) 분석
* 대구와 **비슷한**  지역 기후 탐색
* 대구와 **다른 지형**의 기후 탐색
* HI(열 지수)를 체감 온도로 보고 진행
* 폭염 기준 : 주의보 체감 온도 33℃ 이상 / 경보 35℃ 이상
* 체감 온도: 기온에 습도, 바람 등의 영향이 더해져 사람이 느끼는 더위나 추위를 정량적으로 나타낸 온도
* 습도 10% 증가 시마다 체감온도 1도 가량 증가하는 특징

# 멘토 미팅 결과
* 추가 작업 (주변 지역 탐색) 난이도 문제로 임시적 보류

# 날짜 칼럼 조정 필요
* month , day 합쳐서 date 라는 데이터를 만들면 좋을듯

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 각 도시 데이터 호출
df_daegu = pd.read_csv("output/Daegu_final.csv")
df_daejeon = pd.read_csv("output/Daejeon_final.csv")
df_cheongju = pd.read_csv("output/Cheongju_final.csv")
df_chuncheon = pd.read_csv("output/Chuncheon_final.csv")

# 대구

In [3]:
df_daegu

,year,month,day,hour,datetime,temp,rainfall,lightning,humidity,wind_speed,wind_direction,condition,region,HI,DI
0,2010,6,1,0,2010-06-01 00:00:00,16.5,0.0,-1,53.0,NaN,NaN,1,동인동,15.589444,60.74355
1,2010,6,1,1,2010-06-01 01:00:00,18.1,0.0,-1,49.0,NaN,NaN,1,동인동,17.245000,62.73431
2,2010,6,1,2,2010-06-01 02:00:00,19.7,0.0,-1,46.0,NaN,NaN,1,동인동,18.926667,64.65038
3,2010,6,1,3,2010-06-01 03:00:00,20.8,0.0,-1,41.0,NaN,NaN,1,동인동,20.006111,65.72772
4,2010,6,1,4,2010-06-01 04:00:00,22.2,0.0,-1,36.0,NaN,NaN,2,동인동,21.415556,67.04608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14831611,2022,4,30,19,2022-04-30 19:00:00,10.7,0.0,-1,71.0,2.1,335.0,-1,구지면,9.679444,52.33503
14831612,2022,4,30,20,2022-04-30 20:00:00,11.0,0.0,-1,65.0,2.2,339.0,-1,구지면,9.852778,52.99350
14831613,2022,4,30,21,2022-04-30 21:00:00,11.9,0.0,-1,61.0,2.2,338.0,-1,구지면,10.738333,54.40241
14831614,2022,4,30,22,2022-04-30 22:00:00,13.4,0.0,-1,53.0,2.0,351.0,-1,구지면,12.179444,56.60598


### 칼럼 재조정 및 날짜 보정
* unique 함수 사용을 위해 datetime에 있는 시간 부분을 제거하고 작업

In [4]:
# 칼럼 재조정
daegu = df_daegu[['region','year','datetime','temp','rainfall','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
daegu['datetime'] = daegu['datetime'].str.split(' ').str[0]
daegu

C:\Users\JW\AppData\Local\Temp\ipykernel_3232\2205726566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daegu['datetime'] = daegu['datetime'].str.split(' ').str[0]


,region,year,datetime,temp,rainfall,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,동인동,2010,2010-06-01,16.5,0.0,-1,53.0,NaN,NaN,1,15.589444,60.74355
1,동인동,2010,2010-06-01,18.1,0.0,-1,49.0,NaN,NaN,1,17.245000,62.73431
2,동인동,2010,2010-06-01,19.7,0.0,-1,46.0,NaN,NaN,1,18.926667,64.65038
3,동인동,2010,2010-06-01,20.8,0.0,-1,41.0,NaN,NaN,1,20.006111,65.72772
4,동인동,2010,2010-06-01,22.2,0.0,-1,36.0,NaN,NaN,2,21.415556,67.04608
...,...,...,...,...,...,...,...,...,...,...,...,...
14831611,구지면,2022,2022-04-30,10.7,0.0,-1,71.0,2.1,335.0,-1,9.679444,52.33503
14831612,구지면,2022,2022-04-30,11.0,0.0,-1,65.0,2.2,339.0,-1,9.852778,52.99350
14831613,구지면,2022,2022-04-30,11.9,0.0,-1,61.0,2.2,338.0,-1,10.738333,54.40241
14831614,구지면,2022,2022-04-30,13.4,0.0,-1,53.0,2.0,351.0,-1,12.179444,56.60598


In [5]:
# 조건에 맞는 결과값을 뽑아서 따로 저장 후
# 해당 기간에 대한 고유값만 모아서 저장
# 그 후 그 일자가 모인 리스트의 크기 = 폭염 발생 일 수로 둠

result = []

for i in range(2010,2023):
    daegu_sample = daegu[(daegu['year'] == i) & (daegu['HI'] >= 35) & (daegu['DI'] >= 80)]
    result.append(len(daegu_sample['datetime'].unique()))

print("대구 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("대구 지역 폭염 발생일 평균 : ",round(mean,2))

대구 지역 매년 폭염 발생일 수
[63, 53, 40, 54, 24, 26, 45, 53, 53, 39, 44, 39, 0]
대구 지역 폭염 발생일 평균 :  41.0


# 대전

In [8]:
# 칼럼 재조정
daejeon = df_daejeon[['region','year','datetime','temp','rainfall','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
daejeon['datetime'] = daejeon['datetime'].str.split(' ').str[0]
daejeon

C:\Users\JW\AppData\Local\Temp\ipykernel_3232\1290181901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daejeon['datetime'] = daejeon['datetime'].str.split(' ').str[0]


,region,year,datetime,temp,rainfall,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,중앙동,2010,2010-06-01,17.4,0.0,-1,44.0,NaN,NaN,1,16.344444,61.68144
1,중앙동,2010,2010-06-01,18.7,0.0,-1,39.0,NaN,NaN,1,17.643889,63.09007
2,중앙동,2010,2010-06-01,19.9,0.0,-1,37.0,NaN,NaN,1,18.911667,64.41737
3,중앙동,2010,2010-06-01,20.6,0.0,-1,35.0,NaN,NaN,1,19.629444,65.11890
4,중앙동,2010,2010-06-01,21.7,0.0,-1,33.0,NaN,NaN,1,20.787222,66.24739
...,...,...,...,...,...,...,...,...,...,...,...,...
8251387,목상동,2022,2022-04-30,5.6,0.0,-1,94.0,0.0,0.0,-1,4.670000,42.60536
8251388,목상동,2022,2022-04-30,5.3,0.0,-1,95.0,0.0,230.0,-1,4.366111,41.99265
8251389,목상동,2022,2022-04-30,8.2,0.0,-1,93.0,0.0,0.0,-1,7.503889,47.19274
8251390,목상동,2022,2022-04-30,11.1,0.0,-1,66.0,1.8,200.0,-1,9.988889,53.10574


In [9]:
result = []

for i in range(2010,2023):
    daejeon_sample = daejeon[(daejeon['year'] == i) & (daejeon['HI'] >= 35) & (daejeon['DI'] >= 80)]
    result.append(len(daejeon_sample['datetime'].unique()))

print("대전 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("대전 지역 폭염 발생일 평균 : ",round(mean,2))

대전 지역 매년 폭염 발생일 수
[51, 37, 38, 56, 20, 23, 40, 53, 54, 30, 34, 35, 0]
대전 지역 폭염 발생일 평균 :  36.23


# 청주

In [10]:
# 칼럼 재조정
cheongju = df_cheongju[['region','year','datetime','temp','rainfall','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
cheongju['datetime'] = cheongju['datetime'].str.split(' ').str[0]
cheongju

C:\Users\JW\AppData\Local\Temp\ipykernel_3232\3227368952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cheongju['datetime'] = cheongju['datetime'].str.split(' ').str[0]


,region,year,datetime,temp,rainfall,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,가경동,2010,2010-06-01,17.3,0.0,-1,41.0,NaN,NaN,1,16.156111,61.47207
1,가경동,2010,2010-06-01,19.0,0.0,-1,38.0,NaN,NaN,1,17.947778,63.40380
2,가경동,2010,2010-06-01,20.3,0.0,-1,35.0,NaN,NaN,1,19.299444,64.77195
3,가경동,2010,2010-06-01,21.6,0.0,-1,33.0,NaN,NaN,1,20.677222,66.13372
4,가경동,2010,2010-06-01,22.9,0.0,-1,35.0,NaN,NaN,1,22.159444,67.77885
...,...,...,...,...,...,...,...,...,...,...,...,...
4491259,현도면,2022,2022-04-30,6.9,0.0,-1,92.0,0.5,125.0,-1,6.047778,45.01752
4491260,현도면,2022,2022-04-30,6.6,0.0,-1,94.0,0.6,182.0,-1,5.770000,44.34596
4491261,현도면,2022,2022-04-30,8.3,0.0,-1,91.0,0.8,154.0,-1,7.561667,47.48747
4491262,현도면,2022,2022-04-30,10.8,0.0,-1,74.0,1.1,211.0,-1,9.867778,52.37808


In [11]:
result = []

for i in range(2010,2023):
    cheongju_sample = cheongju[(cheongju['year'] == i) & (cheongju['HI'] >= 35) & (cheongju['DI'] >= 80)]
    result.append(len(cheongju_sample['datetime'].unique()))

print("청주 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("청주 지역 폭염 발생일 평균 : ",round(mean,2))

청주 지역 매년 폭염 발생일 수
[49, 37, 36, 55, 20, 21, 39, 46, 50, 38, 47, 42, 0]
청주 지역 폭염 발생일 평균 :  36.92


# 춘천

In [12]:
# 칼럼 재조정
chuncheon = df_chuncheon[['region','year','datetime','temp','rainfall','lightning','humidity','wind_speed','wind_direction','condition','HI','DI']]
chuncheon['datetime'] = chuncheon['datetime'].str.split(' ').str[0]
chuncheon

C:\Users\JW\AppData\Local\Temp\ipykernel_3232\4166176689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chuncheon['datetime'] = chuncheon['datetime'].str.split(' ').str[0]


,region,year,datetime,temp,rainfall,lightning,humidity,wind_speed,wind_direction,condition,HI,DI
0,강남동,2010,2010-06-01,14.3,0.0,-1,47.0,NaN,NaN,1,13.012778,57.81579
1,강남동,2010,2010-06-01,17.2,0.0,-1,37.0,NaN,NaN,1,15.941667,61.24136
2,강남동,2010,2010-06-01,20.3,0.0,-1,29.0,NaN,NaN,1,19.142778,64.42413
3,강남동,2010,2010-06-01,21.7,0.0,-1,25.0,NaN,NaN,1,20.578333,65.67275
4,강남동,2010,2010-06-01,23.7,0.0,-1,23.0,NaN,NaN,1,22.726111,67.60449
...,...,...,...,...,...,...,...,...,...,...,...,...
2715643,후평3동,2022,2022-04-30,4.2,0.0,-1,97.0,0.4,78.0,-1,4.200000,39.86426
2715644,후평3동,2022,2022-04-30,3.9,0.0,-1,98.0,0.5,317.0,-1,3.900000,39.22878
2715645,후평3동,2022,2022-04-30,5.6,0.0,-1,95.0,0.9,318.0,-1,4.696111,42.51780
2715646,후평3동,2022,2022-04-30,7.7,0.0,-1,85.0,1.1,316.0,-1,6.745000,46.86155


In [13]:
result = []

for i in range(2010,2023):
    chuncheon_sample = chuncheon[(chuncheon['year'] == i) & (chuncheon['HI'] >= 35) & (chuncheon['DI'] >= 80)]
    result.append(len(chuncheon_sample['datetime'].unique()))

print("춘천 지역 매년 폭염 발생일 수" )
print(result)

sum = 0
for i in range(len(result)):
    sum += result[i]

mean = sum / len(result)


print("춘천 지역 폭염 발생일 평균 : ",round(mean,2))

춘천 지역 매년 폭염 발생일 수
[34, 22, 28, 31, 14, 27, 41, 38, 45, 33, 29, 35, 0]
춘천 지역 폭염 발생일 평균 :  29.0


In [17]:
# 작업 완료 후 원본 메모리 할당 해제
del df_daegu
del df_cheongju
del df_chuncheon
del df_daejeon

# 추가) 온열질환자 데이터
* 폭염 피해의 유형 분류 -> 온열질환자 조사 필요성...
* 큰 의미 있는 결과는 아닌듯

In [14]:
df = pd.read_csv("dataset/온열질환.csv")

In [15]:
# 합계 테이블은 실내 실외 소계의 합
df

,연도,지역,합계,실외 소계,실외 실외작업장,실외 운동장,실외 논밭,실외 산,실외 강가해변,실외 길가,실외 주거지주변,실외 기타,실내 소계,실내 집,실내 건물,실내 작업장,실내 비닐하우스,실내 기타
0,2016,서울,170,109,28,9,2,1,1,42,10,16,61,34,8,3,0,16
1,2016,부산,112,71,31,3,3,2,4,17,3,8,41,19,5,14,0,3
2,2016,대구,38,29,5,8,4,2,1,5,2,2,9,4,0,2,0,3
3,2016,인천,102,83,41,6,7,1,1,18,5,4,19,7,1,5,1,5
4,2017,서울,107,83,22,11,1,4,0,22,12,11,24,8,4,4,0,8
5,2017,부산,64,44,17,7,3,0,0,8,6,3,20,10,0,9,0,1
6,2017,대구,28,18,3,1,0,2,0,8,3,1,10,4,1,3,1,1
7,2017,인천,54,43,18,6,2,1,2,5,3,6,11,2,2,3,0,4
8,2018,서울,616,362,104,26,10,11,2,132,42,35,254,198,19,11,0,26
9,2018,부산,208,146,71,8,4,6,4,33,9,11,62,21,3,29,0,9


In [16]:
df[df['지역'] == '대구']

,연도,지역,합계,실외 소계,실외 실외작업장,실외 운동장,실외 논밭,실외 산,실외 강가해변,실외 길가,실외 주거지주변,실외 기타,실내 소계,실내 집,실내 건물,실내 작업장,실내 비닐하우스,실내 기타
2,2016,대구,38,29,5,8,4,2,1,5,2,2,9,4,0,2,0,3
6,2017,대구,28,18,3,1,0,2,0,8,3,1,10,4,1,3,1,1
10,2018,대구,122,89,20,11,8,1,1,28,15,5,33,27,1,2,1,2
14,2019,대구,40,29,4,3,5,1,0,10,3,3,11,5,1,3,1,1
18,2020,대구,26,18,4,1,1,0,0,10,1,1,8,5,1,2,0,0
